In [5]:
import numpy as np
import keras

### load numpy data

In [6]:
# load first datasets
cifer10 = np.load("./loaded_data/cifer10_x_train.npy")
# create labels
cifer10_labels = np.zeros(cifer10.shape[0]-1, dtype=int )
cifer10.shape, cifer10_labels.shape

((20000, 32, 32, 3), (19999,))

In [7]:
# load first faces datasets
faces1 = np.load("./loaded_data/first_10K.npy")
faces1.shape

(9999, 32, 32, 3)

In [8]:
# load second faces datasets
faces2 = np.load("./loaded_data/second_10K.npy")
faces2.shape

(10000, 32, 32, 3)

In [9]:
# concatenate arrays, and create calss labels.
faces = np.concatenate((faces1, faces2), axis=0)
faces_label = np.ones(faces.shape[0], dtype=int)
print(faces.shape, faces_label.shape)

(19999, 32, 32, 3) (19999,)


### make train and evaluate data

In [10]:
# faces
faces_train_data = faces[: 15000]; faces_label_train = faces_label[:15000]
faces_test_data = faces[15000: 17500] ; faces_label_test = faces_label[15000: 17500]
faces_eval_data = faces[17500: ] ; faces_label_eval = faces_label[17500: ]
print( f"Data:\n{faces_train_data.shape},\n{faces_test_data.shape},\n{faces_eval_data.shape}" )
print( f"\nLabels:\n{faces_label_train.shape},\n{faces_label_test.shape},\n{faces_label_eval.shape}" )

Data:
(15000, 32, 32, 3),
(2500, 32, 32, 3),
(2499, 32, 32, 3)

Labels:
(15000,),
(2500,),
(2499,)


In [11]:
# random images
cifer10_train_data = cifer10[: 15000]; cifer10_label_train = cifer10_labels[:15000]
cifer10_test_data = cifer10[15000: 17500] ; cifer10_label_test = cifer10_labels[15000: 17500]
cifer10_eval_data = cifer10[17500: -1] ; cifer10_label_eval = cifer10_labels[17500: ]
print( f"Data:\n{cifer10_train_data.shape},\n{cifer10_test_data.shape},\n{cifer10_eval_data.shape}" )
print( f"\nLabels:\n{cifer10_label_train.shape},\n{cifer10_label_test.shape},\n{cifer10_label_eval.shape}" )

Data:
(15000, 32, 32, 3),
(2500, 32, 32, 3),
(2499, 32, 32, 3)

Labels:
(15000,),
(2500,),
(2499,)


In [12]:
# normalise data
cifer_td = cifer10_train_data / 255.0
cifer_ed = cifer10_eval_data / 255.0
cifer_tsd = cifer10_test_data / 255.0

In [13]:
# concatenate train data
x_train = np.concatenate((faces_train_data, cifer_td), axis=0)
y_train = np.concatenate((faces_label_train, cifer10_label_train ), axis=0)
x_train.shape, y_train.shape

((30000, 32, 32, 3), (30000,))

In [14]:
# concatenate test data
x_test = np.concatenate(( faces_test_data, cifer_tsd ), axis=0)
y_test = np.concatenate(( faces_label_test, cifer10_label_test ), axis=0 )
x_test.shape, y_test.shape

((5000, 32, 32, 3), (5000,))

In [15]:
# concatenate test data
x_eval = np.concatenate((faces_eval_data, cifer10_eval_data ), axis=0)
y_eval = np.concatenate((faces_label_eval, cifer10_label_eval ), axis=0)
x_eval.shape, y_eval.shape

((4998, 32, 32, 3), (4998,))

### normalise datasets and reshape datasets

In [16]:
# reshape datasets
x_train_data = x_train.reshape((x_train.shape[0], -1 ))
# 
x_test_data = x_test.reshape((x_test.shape[0], -1))
# 
x_eval_data = x_eval.reshape(x_eval.shape[0], -1)
# 
print(f"x_train: {x_train_data.shape} \nx_test: {x_test_data.shape} \nx_eval: {x_eval_data.shape}")

x_train: (30000, 3072) 
x_test: (5000, 3072) 
x_eval: (4998, 3072)


In [17]:
print(x_train_data.shape)
x_train[0][0]

(30000, 3072)


array([[0.09411765, 0.09803922, 0.11372549],
       [0.08627451, 0.09019608, 0.10588235],
       [0.0745098 , 0.07843137, 0.09411765],
       [0.06666667, 0.07058824, 0.08627451],
       [0.06666667, 0.07058824, 0.08627451],
       [0.05098039, 0.05490196, 0.07058824],
       [0.03921569, 0.04313725, 0.05098039],
       [0.03921569, 0.04313725, 0.05098039],
       [0.02745098, 0.03137255, 0.03921569],
       [0.02745098, 0.03137255, 0.03921569],
       [0.02745098, 0.03137255, 0.03921569],
       [0.03137255, 0.03529412, 0.04313725],
       [0.02352941, 0.02352941, 0.03137255],
       [0.01568627, 0.01568627, 0.02352941],
       [0.01568627, 0.01568627, 0.01568627],
       [0.01568627, 0.01568627, 0.01176471],
       [0.01568627, 0.01568627, 0.01176471],
       [0.01568627, 0.01176471, 0.01176471],
       [0.01568627, 0.01568627, 0.01176471],
       [0.01568627, 0.01176471, 0.01176471],
       [0.01176471, 0.01176471, 0.01568627],
       [0.01176471, 0.01568627, 0.01568627],
       [0.

In [ ]:
from keras import layers

# Assuming your reshaped data has shape (30000, 32, 32, 3)
input_shape = (32, 32, 3)

# Define the input layer
inputs = keras.Input(shape=input_shape)
# Convert the 3D input to a sequence of 1D vectors
x = layers.Reshape((32*32, 3))(inputs)
# Bidirectional LSTM layer
x = layers.Bidirectional(layers.LSTM(64))(x)
# Dropout layer
x = layers.Dropout(0.5)(x)
# Output layer for binary classification
outputs = layers.Dense(1, activation='sigmoid')(x)
# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Print the model summary
model.summary()


### train model

In [19]:
# Train the model
batch_size = 32
epoch = 10
validation_accuracies = []

for _ in range(5):
    # Train and evaluate the model on the validation set
    model.fit(x_train, y_train, epochs=epoch, batch_size=32, verbose=True, validation_data=(x_eval, y_eval) )
    _, accuracy = model.evaluate(x_eval, y_eval)
    validation_accuracies.append(accuracy)


Epoch 1/10
938/938 [==============================] - 168s 178ms/step - loss: 0.6402 - accuracy: 0.6397 - val_loss: 0.6030 - val_accuracy: 0.7255
Epoch 2/10
938/938 [==============================] - 172s 183ms/step - loss: 0.6818 - accuracy: 0.5753 - val_loss: 0.6151 - val_accuracy: 0.7343
Epoch 3/10
938/938 [==============================] - 182s 194ms/step - loss: 0.6742 - accuracy: 0.5686 - val_loss: 0.6186 - val_accuracy: 0.7369
Epoch 4/10
938/938 [==============================] - 190s 202ms/step - loss: 0.6869 - accuracy: 0.5478 - val_loss: 0.5681 - val_accuracy: 0.6803
Epoch 5/10
938/938 [==============================] - 184s 196ms/step - loss: 0.6792 - accuracy: 0.5691 - val_loss: 0.4880 - val_accuracy: 0.7905
Epoch 6/10
938/938 [==============================] - 186s 198ms/step - loss: 0.6628 - accuracy: 0.5964 - val_loss: 0.4343 - val_accuracy: 0.8631
Epoch 7/10
938/938 [==============================] - 179s 191ms/step - loss: 0.6335 - accuracy: 0.6381 - val_loss: 0.6609 -

In [24]:
from scipy.stats import ttest_1samp

mean_accuracy = np.mean(validation_accuracies)
std_accuracy = np.std(validation_accuracies)

# Perform one-sample t-test
null_hypothesis = 0.5  # Null hypothesis assumes random chance performance
t_statistic, p_value = ttest_1samp(validation_accuracies, null_hypothesis)

# Compare the obtained p-value with the significance level
alpha = 0.05
print( p_value)

0.9959689241794617


In [ ]:

# if p_value < alpha:
#     print("The model's performance is statistically significant.")
# else:
#     print("The model's performance is not statistically significant.")

### using sequential api from keras

In [ ]:
from keras.models import Sequential
from keras.layers import Bidirectional, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Assuming your input shape is (32, 32, 3)
input_shape = (32, 32, 3)
dropout_rate = 0.25

# Create the Sequential model
model = Sequential()
# Add a Bidirectional layer
model.add(Bidirectional(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape)))
# Add MaxPooling2D, Flatten, and Dense layers
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
# Add Dropout layer
model.add(Dropout(dropout_rate))
# Add the final Dense layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

In [21]:
# second model training
# Train the model
model.fit(x_train, y_train, validation_data=(x_eval, y_eval), epochs=10, callbacks=[early_stopping])

NameError: name 'early_stopping' is not defined

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)